<a href="https://colab.research.google.com/github/MJLuan12/app-dev/blob/main/iGlucoseCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from skimage import exposure
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import deeplake

In [10]:
# Constants
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 32
EPOCHS = 50  # Increase epochs for fine-tuning
LEARNING_RATE = 0.0001
CLASS_NAMES = ['0', '1', '2', '3', '4']  # Adjust based on DRD dataset labels

In [11]:
# Preprocessing function
def preprocess_image(image):
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    image = exposure.equalize_adapthist(image, clip_limit=0.03)
    image = preprocess_input(image)  # Apply ResNet50's preprocessing
    return image

In [12]:
# Model Building
def create_model():
    # Load the ResNet50 model with pre-trained ImageNet weights
    image_input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

    # Initially freeze all layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add new layers on top of the ResNet50 output
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(5, activation='softmax')(x)

    # Create the final model
    model = Model(inputs=image_input, outputs=predictions)
    return model

In [13]:
# Model Compilation (initial training)
model = create_model()
model.compile(
    optimizer=AdamW(learning_rate=LEARNING_RATE, weight_decay=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
# Load the DRD datasets from Deep Lake
train_ds = deeplake.load("hub://activeloop/diabetic-retinopathy-detection-train")
test_ds = deeplake.load("hub://activeloop/diabetic-retinopathy-detection-test")

\

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/diabetic-retinopathy-detection-train



/

...

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/diabetic-retinopathy-detection-test



/

hub://activeloop/diabetic-retinopathy-detection-test loaded successfully.



In [29]:
# Get the training labels (assuming the label tensor is named 'diagnosis')
train_labels = train_ds.labels.numpy()  # Convert to NumPy array
train_labels = train_labels.flatten()  # Convert to a 1D array

In [30]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(class_weights))

In [39]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('diabetic_retinopathy_model_best_resnet.keras', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)

In [38]:
def scheduler(epoch, lr):
    return lr * tf.math.exp(-0.1) if epoch >= 10 else lr

lr_scheduler = LearningRateScheduler(scheduler)
callbacks = [early_stopping, checkpoint, reduce_lr, lr_scheduler]

In [36]:
# Define a function to prepare images
def preprocess(image):
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    image = exposure.equalize_adapthist(image, clip_limit=0.03)
    image = preprocess_input(image)  # Apply ResNet50's preprocessing
    return image

In [40]:
# Create TensorFlow data loaders
train_dataloader = train_ds.tensorflow(batch_size=BATCH_SIZE, shuffle=True).map(lambda x: (preprocess(x[0]), x[1]))
test_dataloader = test_ds.tensorflow(batch_size=BATCH_SIZE, shuffle=False).map(lambda x: (preprocess(x[0]), x[1]))

TypeError: Dataset.tensorflow() got an unexpected keyword argument 'batch_size'

In [ ]:
# Training Model (first stage: only train the new layers)
history = model.fit(
    train_dataloader,
    epochs=10,  # Train for a few epochs to get the new layers warmed up
    validation_data=test_dataloader, # Using test set for validation during initial training
    class_weight=class_weights,
    callbacks=callbacks
)

In [ ]:
# Fine-tuning: Unfreeze some ResNet50 layers
for layer in base_model.layers[-20:]:  # Unfreeze the last 20 layers of ResNet50
    layer.trainable = True

In [ ]:
# Re-compile with a lower learning rate
model.compile(
    optimizer=AdamW(learning_rate=LEARNING_RATE / 10, weight_decay=0.01),  # Adjust learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fine-tune the model
history = model.fit(
    train_dataloader,
    epochs=EPOCHS,  # Train for more epochs for fine-tuning
    validation_data=test_dataloader,
    class_weight=class_weights,
    callbacks=callbacks
)

In [ ]:
# Model evaluation (using validation data loader)
validation_images, validation_labels = next(iter(val_dl))  # Get a batch from the validation data loader
predictions = model.predict(validation_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(validation_labels, axis=1)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes, average='macro')
precision = precision_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')
mcc = matthews_corrcoef(true_classes, predicted_classes)
auc_roc = roc_auc_score(validation_labels, predictions, multi_class='ovr')


In [ ]:
# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1 Score: {f1}')
print(f'Matthews Correlation Coefficient: {mcc}')
print(f'AUC-ROC: {auc_roc}')

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Save the model
model.save('diabetic_retinopathy_model_finetuned_resnet.h5')